In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta

In [3]:
# Get yesterday’s date
yesterday = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')

In [4]:
# MLB API endpoint for daily schedule
url = f'https://statsapi.mlb.com/api/v1/schedule?sportId=1&date={yesterday}'
print(url)

https://statsapi.mlb.com/api/v1/schedule?sportId=1&date=2025-07-05


In [9]:
response = requests.get(url)
data = response.json()
with open(f'assets/json/boxscores/mlb_boxscores_{yesterday}.json', 'w') as f:
    json.dump(data, f, indent=2)

# Extract games list
games = data.get('dates', [{}])[0].get('games', [])

if games:
    df = pd.json_normalize(games)
    # Filter games where status.detailedState is 'Final'
    df = df[df['status.detailedState'] == 'Final']
    df = df[['gamePk', 'gameDate', 'teams.away.team.id', 'teams.home.team.id']]
else:
    df = pd.DataFrame()
df.to_csv(f'mlb_games_{yesterday}.csv', index=False)

In [7]:
# Read the CSV file with gamePk values
games_df = pd.read_csv('mlb_games_2025-07-05.csv')

boxscores = []
for game_pk in games_df['gamePk']:
    url = f'https://statsapi.mlb.com/api/v1/game/{game_pk}/boxscore'
    response = requests.get(url)
    if response.status_code == 200:
        boxscore_data = response.json()
        # You can customize what to extract; here we just store the full boxscore
        boxscores.append({'gamePk': game_pk, 'boxscore': boxscore_data})
    else:
        boxscores.append({'gamePk': game_pk, 'boxscore': None})

# Optionally, save all boxscores to a JSON file
with open('assets/json/boxscores/mlb_boxscores_2025-07-05.json', 'w') as f:
    json.dump(boxscores, f)

In [10]:
# ...existing code...

for game in boxscores:
    boxscore = game['boxscore']
    if not boxscore:
        continue

    for team_type in ['home', 'away']:
        team = boxscore['teams'][team_type]
        batters = team.get('batters', [])
        players = team.get('players', {})

        print(f"\n{team_type.upper()} TEAM BATTERS:")
        for batter_id in batters:
            player_key = f"ID{batter_id}"
            player = players.get(player_key)
            if player:
                name = player['person']['fullName']
                batting_stats = player.get('stats', {}).get('batting', {})
                summary = batting_stats.get('summary', 'No stats')
                print(f"{batter_id}: {name} - {summary}")
            else:
                print(f"{batter_id}: Player data not found")

# ...existing code...


HOME TEAM BATTERS:
621439: Byron Buxton - 0-4 | BB, K, R
650489: Willi Castro - 1-4 | BB, K
663616: Trevor Larnach - 1-3 | 2B
686797: Brooks Lee - 1-2 | RBI
621043: Carlos Correa - 2-4 | 2 R
680777: Ryan Jeffers - 3-4 | R
670242: Matt Wallner - 0-2 | BB, K
664034: Ty France - 0-1 | K
668904: Royce Lewis - 2-4 | 2 RBI, R
665019: Kody Clemens - 1-4 | HR, 3 K, 3 RBI
664056: Harrison Bader - 0-3 | BB
663485: Cole Sands - No stats
543056: Danny Coulombe - No stats
701519: Travis Adams - No stats
643377: Griffin Jax - No stats
661395: Jhoan Duran - No stats

AWAY TEAM BATTERS:
650490: Yandy Díaz - 0-4 | K, RBI
664040: Brandon Lowe - 1-4 | BB, K, SB
691406: Junior Caminero - 0-4 | K, RBI
666018: Jonathan Aranda - 1-4 | 2B, 2 K, R
663968: Jake Mangum - 2-4 | 2B, R
666139: Josh Lowe - 1-3 | 2B, BB, K
670764: Taylor Walls - 1-4 | K
802415: Chandler Simpson - 2-3 | RBI, 2 R
643376: Danny Jansen - 3-3 | 2B, BB, RBI
676609: José Caballero - 0-0 | SB
642136: Matt Thaiss - 0-0
671737: Taj Bradley - 